In [18]:
import ipywidgets as widgets
from IPython.display import clear_output, display
import requests
import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots



# Create heading
heading = widgets.HTML('<h1>App: Energy Market Simulator</h1>')

# Create layout and style
layout = widgets.Layout(width='150px', height='25px', margin='5px 0 0 0', background_color='lightgray', color='black')  # Set a fixed width and colors
button_style = widgets.ButtonStyle(button_color='lightblue', font_weight='bold')
csv_path_layout = widgets.Layout(width='300px', height='25px', margin='5px 0 0 0', background_color='lightgray', color='black')  # Set a fixed width and colors

# Create widgets with layout
battery_capacity = widgets.FloatSlider(min=1, max=50.0, step=1.0, value=1.0, layout=layout)
charge_efficiency = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.9, layout=layout)
discharge_efficiency = widgets.FloatSlider(min=0.1, max=1, step=0.1, value=0.9, layout=layout)
start_date = widgets.DatePicker(value=datetime.date(2015, 2, 1), layout=layout)
end_date = widgets.DatePicker(value=datetime.date(2015, 2, 2), layout=layout)
price_model = widgets.Dropdown(options=['SimulatedPriceModel','HistoricalPriceModel'], value='SimulatedPriceModel', layout=layout)
# Create csv_path with new layout
csv_path = widgets.Text(value='data/time_series/time_series_60min_singleindex_filtered.csv', layout=csv_path_layout, disabled=True)

# Function to observe changes in price_model
def on_price_model_change(change):
    if change['new'] == 'HistoricalPriceModel':
        csv_path.disabled = False
    else:
        csv_path.disabled = True

# Observe changes in price_model
price_model.observe(on_price_model_change, names='value')


# Create labels
battery_capacity_label = widgets.Label('Battery Capacity', layout=layout)
charge_efficiency_label = widgets.Label('Charge Efficiency', layout=layout)
discharge_efficiency_label = widgets.Label('Discharge Efficiency', layout=layout)
start_date_label = widgets.Label('Start Date', layout=layout)
end_date_label = widgets.Label('End Date', layout=layout)
price_model_label = widgets.Label('Price Model', layout=layout)
csv_path_label = widgets.Label('CSV Path', layout=layout)

# Create button with style
button = widgets.Button(description="Run Simulation", style=button_style, layout=layout)

# Create layout for Boxes
box_layout = widgets.Layout(display='flex', flex_flow='row wrap', align_items='flex-start', width='auto')

# Create spacers
spacer = widgets.HTML(value="&nbsp;"*5)

# Create Boxes with box_layout and spacers
first_line = widgets.Box([battery_capacity_label, battery_capacity, charge_efficiency_label, charge_efficiency, discharge_efficiency_label, discharge_efficiency], layout=box_layout)
second_line = widgets.Box([start_date_label, start_date, spacer, end_date_label, end_date], layout=box_layout)
third_line = widgets.Box([price_model_label, price_model, spacer, csv_path_label, csv_path], layout=box_layout)

# Create an output widget
out = widgets.Output()

# Create a new figure widget
fig = go.FigureWidget(make_subplots(rows=2, cols=1))

# Display HBoxes, button, and output widget
display(heading, first_line, second_line, third_line, button, out, fig)


def on_button_clicked(b):

    # Clear output
    fig.data = []
    
    # Send GET request
    response = requests.get('http://localhost:5000/simulate', params={
        'battery_capacity': battery_capacity.value,
        'charge_efficiency': charge_efficiency.value,
        'discharge_efficiency': discharge_efficiency.value,
        'start_date': start_date.value,
        'end_date': end_date.value,
        'price_model': price_model.value,
        'csv_path': csv_path.value
    })

    # Convert response to JSON
    data = response.json()

    # Get unique dates
    unique_dates = sorted(set(item['current_date'] for item in data))
    
    # Create a trace for each date
    for date in unique_dates:
        for item in data:
            # Check if the current item's date matches the current date
            if item['current_date'] == date:
                # Get the data for the current date
                schedule_df = item['schedule_df']
                daily_pnl = item['daily_pnl']

                # Convert Interval, Charge, Discharge, and SOC to lists
                interval = list(schedule_df['Interval'].values())
                charge = list(schedule_df['Charge'].values())
                discharge = list(schedule_df['Discharge'].values())
                soc = list(schedule_df['SOC'].values())

                # Add bar plot for Charge and -1*Discharge to the first subplot
                fig.add_trace(go.Bar(name='Charge', x=interval, y=charge, visible=False, marker=dict(color='blue')), row=1, col=1)
                fig.add_trace(go.Bar(name='Discharge', x=interval, y=[-1*i for i in discharge], visible=False, marker=dict(color='red')), row=1, col=1)

                # Add line plot for SOC to the second subplot
                fig.add_trace(go.Scatter(name='SOC', x=interval, y=soc, mode='lines', visible=False, line=dict(color='green')), row=2, col=1)
                                

    # Make the traces and annotation for the first date visible
    for i in range(3):  # Update this line
        fig.data[i].visible = True

    # Create a dropdown menu with one button for each date
    buttons = []
    for i, date in enumerate(unique_dates):
        # Get the daily_pnl for the current date
        daily_pnl = next(item['daily_pnl'] for item in data if item['current_date'] == date)

        visible = [i==j for j in range(len(unique_dates))]
        button = dict(
            label = str(date),
            method = 'update',
            args = [{'visible': visible},
                    {'title': f"Charge/Discharge Action and SOC for {date}<br><sub>Daily PnL: {daily_pnl}</sub>"}])
        buttons.append(button)

    # Add the dropdown menu to the plot
    fig.update_layout(
        updatemenus=[
            dict(
                active=0,
                buttons=buttons,
            )
        ]
    )
    # Get the daily_pnl for the first date
    first_date_pnl = next(item['daily_pnl'] for item in data if item['current_date'] == unique_dates[0])

    # Update layout
    fig.update_layout(height=600, width=800, 
                    title_text=f"Charge/Discharge Action and SOC for {unique_dates[0]}<br><sub>Daily PnL: {first_date_pnl}</sub>")




button.on_click(on_button_clicked)

HTML(value='<h1>App: Energy Market Simulator</h1>')

Box(children=(Label(value='Battery Capacity', layout=Layout(height='25px', margin='5px 0 0 0', width='150px'))…

Box(children=(Label(value='Start Date', layout=Layout(height='25px', margin='5px 0 0 0', width='150px')), Date…

Box(children=(Label(value='Price Model', layout=Layout(height='25px', margin='5px 0 0 0', width='150px')), Dro…

Button(description='Run Simulation', layout=Layout(height='25px', margin='5px 0 0 0', width='150px'), style=Bu…

Output()

FigureWidget({
    'data': [],
    'layout': {'template': '...',
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0]},
               'xaxis2': {'anchor': 'y2', 'domain': [0.0, 1.0]},
               'yaxis': {'anchor': 'x', 'domain': [0.575, 1.0]},
               'yaxis2': {'anchor': 'x2', 'domain': [0.0, 0.425]}}
})